In [ ]:
import json
from pathlib import Path

# =========================================================
# AI Hub 곤충 데이터셋 JSON -> YOLO 포맷 변환 스크립트
# 이미지 해상도를 기준으로 좌표를 정규화하여 .txt 라벨 생성
# =========================================================

# 이미지 원본 해상도 설정 (AI Hub 곤충 데이터 기준 1920x1080)
IMG_W, IMG_H = 1920, 1080

# 입력 데이터 경로: '라벨링데이터' 상위 폴더
# 주의: 본인의 환경에 맞게 경로를 수정해야 할 수 있습니다.
ROOT = Path(r"C:\Users\USER\Desktop\파이썬 기반 딥러닝 프로젝트\064.식용곤충 사육 자동화 데이터\3.개방데이터\1.데이터\Training\02.라벨링데이터")

# 출력 데이터 경로: 변환된 라벨(.txt)이 저장될 위치
OUT_ROOT = Path(r"C:\dataset\mealworm_yolo\labels")
# 출력 디렉토리가 없으면 생성 (상위 경로 포함)
OUT_ROOT.mkdir(parents=True, exist_ok=True)

for json_file in ROOT.rglob("*.json"):
    try:
        # 원본 상위 폴더명 추출 (예: TL_R_BOX_MB_NM)
        # 데이터셋의 구조를 유지하기 위해 상위 폴더명을 그대로 사용
        top_folder = json_file.relative_to(ROOT).parts[0]

        # 최종 저장 경로: C:\dataset\mealworm_yolo\labels\TL_R_BOX_MB_NM
        out_dir = OUT_ROOT / top_folder
        out_dir.mkdir(parents=True, exist_ok=True)

        # JSON 파일 로드
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        # 파일명 추출 (확장자 제외)
        filename = Path(data["info"]["filename"]).stem
        anns = data.get("annotation", [])

        lines = []
        for ann in anns:
            p = ann["points"]
            # 객체 상태(object_status)에 따른 클래스 구분
            # NM(정상) -> 0, 그 외(AB, NG 등) -> 1
            status = ann.get("object_status", "NM")
            cls = 0 if status == "NM" else 1

            # YOLO 포맷 좌표 계산 (정규화된 중심좌표 x, y와 너비 w, 높이 h)
            x_center = (p["x"] + p["width"]/2) / IMG_W
            y_center = (p["y"] + p["height"]/2) / IMG_H
            w = p["width"] / IMG_W
            h = p["height"] / IMG_H
            
            # 결과 리스트에 추가 (소수점 6자리)
            lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

        # 변환된 라벨 파일(.txt) 저장
        out_path = out_dir / f"{filename}.txt"
        out_path.write_text("\n".join(lines), encoding="utf-8")

    except Exception as e:
        print(f"[오류] {json_file} -> {e}")

print("변환 완료! 라벨이 상위 폴더별로 저장되었습니다:", OUT_ROOT)
